In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import random
import copy
import time
import pandas as pd
import numpy as np
import gc
import re
import torch
from torchtext import data

from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')
from collections import Counter
from textblob import TextBlob
from nltk import word_tokenize

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable

from sklearn.metrics import f1_score
import torchtext
import os

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from torch.optim.optimizer import Optimizer

from sklearn.preprocessing import StandardScaler
from textblob import TextBlob
from multiprocessing import  Pool
from functools import partial
import numpy as np
from sklearn.decomposition import PCA
import torch as t
import torch.nn as nn
import torch.nn.functional as F

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [4]:
embed_size = 200
max_features = 99001
maxlen = 70
batch_size = 512
n_epochs = 5
n_splits = 5
SEED = 10
debug = 0

In [5]:
loss_fn = torch.nn.BCEWithLogitsLoss(reduction='sum')

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, f' {punct} ')
    return x


def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

mispell_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot",
                "'cause": "because", "could've": "could have",
                "couldn't": "could not", "didn't": "did not",
                "doesn't": "does not", "don't": "do not", "hadn't": "had not",
                "hasn't": "has not", "haven't": "have not", "he'd": "he would",
                "he'll": "he will", "he's": "he is", "how'd": "how did",
                "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                "I'd": "I would", "I'd've": "I would have", "I'll": "I will",
                "I'll've": "I will have","I'm": "I am", "I've": "I have",
                "i'd": "i would", "i'd've": "i would have", "i'll": "i will",
                "i'll've": "i will have","i'm": "i am", "i've": "i have",
                "isn't": "is not", "it'd": "it would", "it'd've": "it would have",
                "it'll": "it will", "it'll've": "it will have","it's": "it is",
                "let's": "let us", "ma'am": "madam", "mayn't": "may not",
                "might've": "might have","mightn't": "might not",
                "mightn't've": "might not have", "must've": "must have",
                "mustn't": "must not", "mustn't've": "must not have",
                "needn't": "need not", "needn't've": "need not have",
                "o'clock": "of the clock", "oughtn't": "ought not",
                "oughtn't've": "ought not have", "shan't": "shall not",
                "sha'n't": "shall not", "shan't've": "shall not have",
                "she'd": "she would", "she'd've": "she would have",
                "she'll": "she will", "she'll've": "she will have",
                "she's": "she is", "should've": "should have",
                "shouldn't": "should not", "shouldn't've": "should not have",
                "so've": "so have","so's": "so as", "this's": "this is",
                "that'd": "that would", "that'd've": "that would have",
                "that's": "that is", "there'd": "there would",
                "there'd've": "there would have", "there's": "there is",
                "here's": "here is","they'd": "they would",
                "they'd've": "they would have", "they'll": "they will",
                "they'll've": "they will have", "they're": "they are",
                "they've": "they have", "to've": "to have",
                "wasn't": "was not", "we'd": "we would", "we'd've": "we would have",
                "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                "we've": "we have", "weren't": "were not", "what'll": "what will",
                "what'll've": "what will have", "what're": "what are",
                "what's": "what is", "what've": "what have", "when's": "when is",
                "when've": "when have", "where'd": "where did", "where's": "where is",
                "where've": "where have", "who'll": "who will",
                "who'll've": "who will have", "who's": "who is",
                "who've": "who have", "why's": "why is", "why've": "why have",
                "will've": "will have", "won't": "will not", "won't've": "will not have",
                "would've": "would have", "wouldn't": "would not",
                "wouldn't've": "would not have", "y'all": "you all",
                "y'all'd": "you all would","y'all'd've": "you all would have",
                "y'all're": "you all are","y'all've": "you all have",
                "you'd": "you would", "you'd've": "you would have",
                "you'll": "you will", "you'll've": "you will have",
                "you're": "you are", "you've": "you have", 'colour': 'color',
                'centre': 'center', 'favourite': 'favorite',
                'travelling': 'traveling', 'counselling': 'counseling',
                'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor',
                'organisation': 'organization', 'wwii': 'world war 2',
                'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora',
                'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist',
                'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can',
                'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do',
                'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does'}


def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

In [9]:
def load_and_prec():

    train_df = pd.read_csv("/content/drive/MyDrive/clean_text.csv")[:186672]
    test_df = pd.read_csv("/content/drive/MyDrive/clean_text.csv")[186673:]

    print("Train shape : ",train_df.shape)
    print("Test shape : ",test_df.shape)

    train_df["clean_text"] = train_df["clean_text"].apply(lambda x: x.lower() if isinstance(x, str) else x)
    test_df["clean_text"] = test_df["clean_text"].apply(lambda x: x.lower() if isinstance(x, str) else x)

    train_df["clean_text"] = train_df["clean_text"].progress_apply(lambda x: clean_text(x) if isinstance(x, str) else x)
    test_df["clean_text"] = test_df["clean_text"].apply(lambda x: clean_text(x) if isinstance(x, str) else x)

    train_df["clean_text"] = train_df["clean_text"].progress_apply(lambda x: clean_numbers(x) if isinstance(x, str) else x)
    test_df["clean_text"] = test_df["clean_text"].apply(lambda x: clean_numbers(x) if isinstance(x, str) else x)

    train_df["clean_text"] = train_df["clean_text"].progress_apply(lambda x: replace_typical_misspell(x) if isinstance(x, str) else x)
    test_df["clean_text"] = test_df["clean_text"].apply(lambda x: replace_typical_misspell(x) if isinstance(x, str) else x)

    train_X = train_df["clean_text"].fillna("_##_").values
    test_X = test_df["clean_text"].fillna("_##_").values

    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(train_X))
    train_X = tokenizer.texts_to_sequences(train_X)
    test_X = tokenizer.texts_to_sequences(test_X)

    train_X = pad_sequences(train_X, maxlen=maxlen)
    test_X = pad_sequences(test_X, maxlen=maxlen)

    train_df['class'] = train_df['class'].map({'suicide': 1, 'non-suicide': 0})

    train_y = train_df['class'].values

    np.random.seed(SEED)
    trn_idx = np.random.permutation(len(train_X))

    train_X = train_X[trn_idx]
    train_y = train_y[trn_idx]
    return train_X, test_X, train_y, tokenizer.word_index

In [10]:
x_train, x_test, y_train, word_index = load_and_prec()

Train shape :  (186672, 3)
Test shape :  (45401, 3)


Progress:   0%|          | 0/186672 [00:00<?, ?it/s]

Progress:   0%|          | 0/186672 [00:00<?, ?it/s]

Progress:   0%|          | 0/186672 [00:00<?, ?it/s]

In [11]:
def load_glove(word_index):
    EMBEDDING_FILE = '/content/drive/MyDrive/glove.twitter.27B.200d.txt'
    def get_coefs(word,*arr):
      return word, np.asarray(arr, dtype='float32')[:200]
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    nb_words = min(max_features, len(word_index))

    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features:
          continue
        embedding_vector = embeddings_index.get(word)

        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        else:
            embedding_vector = embeddings_index.get(word.capitalize())
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector

    return embedding_matrix

In [12]:
glove_embeddings = load_glove(word_index)
np.shape(glove_embeddings)

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


(99001, 200)

In [13]:
class MyDataset(Dataset):
    def __init__(self,dataset):
        self.dataset = dataset
    def __getitem__(self,index):
        data,target = self.dataset[index]
        return data,target,index
    def __len__(self):
        return len(self.dataset)

In [14]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

x_test_cuda = torch.tensor(x_test, dtype=torch.long).cuda()
test = torch.utils.data.TensorDataset(x_test_cuda)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)

In [15]:
def pytorch_model_run_cv(x_train,y_train,features,x_test, model_obj, feats = False,clip = True):
    avg_losses_f = []
    avg_val_losses_f = []

    train_preds = np.zeros((len(x_train)))

    test_preds = np.zeros((len(x_test)))
    splits = list(StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED).split(x_train, y_train))
    for i, (train_idx, valid_idx) in enumerate(splits):
        x_train = np.array(x_train)
        y_train = np.array(y_train)
        if feats:
            features = np.array(features)
        x_train_fold = torch.tensor(x_train[train_idx.astype(int)], dtype=torch.long).cuda()
        y_train_fold = torch.tensor(y_train[train_idx.astype(int), np.newaxis], dtype=torch.float32).cuda()
        if feats:
            kfold_X_features = features[train_idx.astype(int)]
            kfold_X_valid_features = features[valid_idx.astype(int)]
        x_val_fold = torch.tensor(x_train[valid_idx.astype(int)], dtype=torch.long).cuda()
        y_val_fold = torch.tensor(y_train[valid_idx.astype(int), np.newaxis], dtype=torch.float32).cuda()

        model = copy.deepcopy(model_obj)

        model.cuda()

        loss_fn = torch.nn.BCEWithLogitsLoss(reduction='sum')
        optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),
                                 lr=0.001)

        ################################################################################################
        scheduler = False
        ###############################################################################################

        train = MyDataset(torch.utils.data.TensorDataset(x_train_fold, y_train_fold))
        valid = MyDataset(torch.utils.data.TensorDataset(x_val_fold, y_val_fold))

        train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
        valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)

        print(f'Fold {i + 1}')
        for epoch in range(n_epochs):
            start_time = time.time()
            model.train()

            avg_loss = 0.
            for i, (x_batch, y_batch, index) in enumerate(train_loader):
                if feats:
                    f = kfold_X_features[index]
                    y_pred = model([x_batch,f])
                else:
                    y_pred = model(x_batch)

                if scheduler:
                    scheduler.batch_step()

                loss = loss_fn(y_pred, y_batch)
                optimizer.zero_grad()
                loss.backward()
                if clip:
                    nn.utils.clip_grad_norm_(model.parameters(),1)
                optimizer.step()
                avg_loss += loss.item() / len(train_loader)

            model.eval()

            valid_preds_fold = np.zeros((x_val_fold.size(0)))
            test_preds_fold = np.zeros((len(x_test)))

            avg_val_loss = 0.
            for i, (x_batch, y_batch,index) in enumerate(valid_loader):
                if feats:
                    f = kfold_X_valid_features[index]
                    y_pred = model([x_batch,f]).detach()
                else:
                    y_pred = model(x_batch).detach()

                avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
                valid_preds_fold[index] = sigmoid(y_pred.cpu().numpy())[:, 0]

            elapsed_time = time.time() - start_time
            print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t time={:.2f}s'.format(
                epoch + 1, n_epochs, avg_loss, avg_val_loss, elapsed_time))
        avg_losses_f.append(avg_loss)
        avg_val_losses_f.append(avg_val_loss)

        for i, (x_batch,) in enumerate(test_loader):
            if feats:
                f = test_features[i * batch_size:(i+1) * batch_size]
                y_pred = model([x_batch,f]).detach()
            else:
                y_pred = model(x_batch).detach()

            test_preds_fold[i * batch_size:(i+1) * batch_size] = sigmoid(y_pred.cpu().numpy())[:, 0]

        train_preds[valid_idx] = valid_preds_fold
        test_preds += test_preds_fold / len(splits)

    print('All \t loss={:.4f} \t val_loss={:.4f} \t '.format(np.average(avg_losses_f),np.average(avg_val_losses_f)))
    return train_preds, test_preds, model

In [28]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.hidden_size = 64
        drp = 0.1
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(glove_embeddings, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.conv1d = nn.Conv1d(embed_size, self.hidden_size, kernel_size=3, padding=1)
        self.maxpool = nn.MaxPool1d(kernel_size=2)
        self.linear = nn.Linear(self.hidden_size, 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drp)
        self.out = nn.Linear(64, 1)

    def forward(self, x):
        h_embedding = self.embedding(x)
        h_embedding = h_embedding.permute(0, 2, 1)
        h_conv = self.conv1d(h_embedding)
        h_pool = self.maxpool(h_conv)
        h_pool = h_pool.view(h_pool.size(0), -1)
        h_linear = self.linear(h_pool)
        h_relu = self.relu(h_linear)
        h_dropout = self.dropout(h_relu)
        out = self.out(h_dropout)
        return out


In [32]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.hidden_size = 64
        drp = 0.1
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(glove_embeddings, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.rnn = nn.RNN(embed_size, self.hidden_size, batch_first=True)
        self.linear = nn.Linear(self.hidden_size*2, 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drp)
        self.out = nn.Linear(64, 1)

    def forward(self, x):
        h_embedding = self.embedding(x)
        h_rnn, _ = self.rnn(h_embedding)
        avg_pool = torch.mean(h_rnn, 1)
        max_pool, _ = torch.max(h_rnn, 1)
        conc = torch.cat((avg_pool, max_pool), 1)
        conc = self.relu(self.linear(conc))
        conc = self.dropout(conc)
        out = self.out(conc)
        return out


In [35]:
def bestThreshold(y_train,train_preds):
    tmp = [0,0,0] # idx, cur, max
    delta = 0
    for tmp[0] in tqdm(np.arange(0.1, 0.501, 0.01)):
        tmp[1] = f1_score(y_train, np.array(train_preds)>tmp[0])
        if tmp[1] > tmp[2]:
            delta = tmp[0]
            tmp[2] = tmp[1]
    print('best threshold is {:.4f} with F1 score: {:.4f}'.format(delta, tmp[2]))
    return delta , tmp[2]

In [29]:
# For CNN
train_preds, test_preds, model = pytorch_model_run_cv(x_train,y_train,None,x_test,CNN(), feats = False, clip=False)

Fold 1
Epoch 1/5 	 loss=163.6151 	 val_loss=123.9705 	 time=3.78s
Epoch 2/5 	 loss=105.5533 	 val_loss=100.5537 	 time=3.80s
Epoch 3/5 	 loss=92.3674 	 val_loss=100.9840 	 time=4.18s
Epoch 4/5 	 loss=84.3000 	 val_loss=90.9552 	 time=3.41s
Epoch 5/5 	 loss=77.8738 	 val_loss=97.6561 	 time=3.65s
Fold 2
Epoch 1/5 	 loss=162.6815 	 val_loss=116.0747 	 time=4.26s
Epoch 2/5 	 loss=104.6626 	 val_loss=100.5894 	 time=3.76s
Epoch 3/5 	 loss=91.4802 	 val_loss=96.3580 	 time=3.65s
Epoch 4/5 	 loss=82.9578 	 val_loss=91.2820 	 time=3.45s
Epoch 5/5 	 loss=76.8608 	 val_loss=89.8309 	 time=4.32s
Fold 3
Epoch 1/5 	 loss=163.6791 	 val_loss=113.2737 	 time=3.92s
Epoch 2/5 	 loss=106.1263 	 val_loss=99.5870 	 time=3.42s
Epoch 3/5 	 loss=92.9323 	 val_loss=94.5191 	 time=3.66s
Epoch 4/5 	 loss=84.5883 	 val_loss=93.6004 	 time=4.14s
Epoch 5/5 	 loss=77.6549 	 val_loss=89.4188 	 time=3.85s
Fold 4
Epoch 1/5 	 loss=162.3981 	 val_loss=122.5041 	 time=3.48s
Epoch 2/5 	 loss=104.9130 	 val_loss=96.7156 	

In [30]:
# For CNN
delta, _ = bestThreshold(y_train,train_preds)

  0%|          | 0/41 [00:00<?, ?it/s]

best threshold is 0.4800 with F1 score: 0.9327


In [33]:
# For RNN
train_preds, test_preds, model = pytorch_model_run_cv(x_train,y_train,None,x_test,RNN(), feats = False, clip=False)

Fold 1
Epoch 1/5 	 loss=161.1044 	 val_loss=115.0651 	 time=3.74s
Epoch 2/5 	 loss=104.8252 	 val_loss=99.8669 	 time=3.65s
Epoch 3/5 	 loss=91.8859 	 val_loss=95.6190 	 time=3.84s
Epoch 4/5 	 loss=83.7366 	 val_loss=92.1483 	 time=4.17s
Epoch 5/5 	 loss=77.5403 	 val_loss=89.0532 	 time=3.42s
Fold 2
Epoch 1/5 	 loss=161.9197 	 val_loss=115.8022 	 time=3.69s
Epoch 2/5 	 loss=104.3457 	 val_loss=100.9133 	 time=3.98s
Epoch 3/5 	 loss=90.7609 	 val_loss=97.3698 	 time=4.05s
Epoch 4/5 	 loss=83.0176 	 val_loss=91.6561 	 time=3.39s
Epoch 5/5 	 loss=77.0813 	 val_loss=92.9281 	 time=3.64s
Fold 3
Epoch 1/5 	 loss=163.1619 	 val_loss=113.2580 	 time=4.09s
Epoch 2/5 	 loss=105.4152 	 val_loss=99.5644 	 time=3.91s
Epoch 3/5 	 loss=91.8172 	 val_loss=90.2529 	 time=3.39s
Epoch 4/5 	 loss=82.7735 	 val_loss=87.7881 	 time=3.67s
Epoch 5/5 	 loss=76.6219 	 val_loss=87.4034 	 time=4.43s
Fold 4
Epoch 1/5 	 loss=162.3367 	 val_loss=115.1733 	 time=3.43s
Epoch 2/5 	 loss=106.1227 	 val_loss=107.7302 	 

In [36]:
# For RNN
delta, _ = bestThreshold(y_train,train_preds)

  0%|          | 0/41 [00:00<?, ?it/s]

best threshold is 0.4900 with F1 score: 0.9333
